In [2]:
%pip install pandas // install pandas for code 
%pip install scikit-learn // install sckit-learn 
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import joblib
import numpy as np



#loading the uploaded datasets
# you may need to change this to run the code 
traversal_cost_data_pathway = '/Users/iwuan/OneDrive - hull.ac.uk/_GDrive/Artificial intelligence coursework/AI coursework/traversal_cost_data.csv'

traversal_cost_data = pd.read_csv(traversal_cost_data_pathway) 

# Displays the first rows of the data 
traversal_cost_data.head()

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '//': Expected package name at the start of dependency specifier
    //
    ^


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '//': Expected package name at the start of dependency specifier
    //
    ^


,elevation,avg_traffic_speed,pollution_level,population_density,road_quality_index,proximity_main_roads,weather_condition_index,type_of_terrain,zone_classification,time_of_day,traversal_cost
0,37.454012,72.999831,89.673612,3707.776016,8.363480,580.779043,2.167786,Park,Commercial,Morning,569.106430
1,95.071431,18.451200,28.445333,4405.509375,2.307428,526.971650,5.857533,Park,Residential,Afternoon,378.245893
2,73.199394,34.663969,37.907767,2315.899377,9.518174,351.036950,6.538477,Residential Area,Residential,Morning,534.402743
3,59.865848,66.328064,54.201338,1445.893658,8.589017,493.212658,5.569347,Road,Residential,Evening,826.217263
4,15.601864,48.208934,61.096000,1594.232872,9.269680,365.096635,5.658375,Park,Residential,Morning,186.537161


In [12]:
# Identifying non-numerical features
non_numerical_data = ['type_of_terrain', 'zone_classification', 'time_of_day']

# Checking for missing values in the dataset
missing_values = traversal_cost_data.isnull().sum()

# Encoding non-numerical features
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_data = encoder.fit_transform(traversal_cost_data[non_numerical_data])

# Creating a data frame for the encoded data
encoded_feature_names = encoder.get_feature_names_out(non_numerical_data)
encoded_features_df = pd.DataFrame(encoded_data, columns=encoded_feature_names)

# Dropping non-numerical features from the original dataset and concatenating the encoded features
numerical_features = traversal_cost_data.drop(columns=non_numerical_data + ['traversal_cost'])
processed_data = pd.concat([numerical_features, encoded_features_df, traversal_cost_data['traversal_cost']], axis=1)


# Splitting the data into training and testing sets (It is an 80-20 split)
X = processed_data.drop(columns=['traversal_cost'])
y = processed_data['traversal_cost']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# A summary of preprocessing steps
{
    "missing_values": missing_values.to_dict(),
    "encoded_feature_names": list(encoded_feature_names),
    "train_test_split": {"X_train_shape": X_train.shape, "X_test_shape": X_test.shape}
}
#this outputs all the encoded data names, for example for the "type of terrain" has three new columns park, residentail area and road..
#train_test_split contain x_train_shape which is a set of 16,000 nsamples with 15 features each 
#and x_test_shape contains 4000 samples with 15 features each ... this confirms that the dataset  has a 80-20 split 

{'missing_values': {'elevation': 0,
  'avg_traffic_speed': 0,
  'pollution_level': 0,
  'population_density': 0,
  'road_quality_index': 0,
  'proximity_main_roads': 0,
  'weather_condition_index': 0,
  'type_of_terrain': 0,
  'zone_classification': 0,
  'time_of_day': 0,
  'traversal_cost': 0},
 'encoded_feature_names': ['type_of_terrain_Park',
  'type_of_terrain_Residential Area',
  'type_of_terrain_Road',
  'zone_classification_Industrial',
  'zone_classification_Residential',
  'time_of_day_Evening',
  'time_of_day_Morning',
  'time_of_day_Night'],
 'train_test_split': {'X_train_shape': (16000, 15),
  'X_test_shape': (4000, 15)}}

In [13]:
# Training the Linear Regression model

linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train_scaled, y_train)

# Predicting the test set
y_pred_linear = linear_regression_model.predict(X_test_scaled)

# Evaluating the model
mae_linear = mean_absolute_error(y_test, y_pred_linear)
mse_linear = mean_squared_error(y_test, y_pred_linear)
rmse_linear = np.sqrt(mse_linear)

# Compiling the results
linear_regression_metrics = {
    "Mean Absolute Error (MAE)": mae_linear,
    "Mean Squared Error (MSE)": mse_linear,
    "Root Mean Squared Error (RMSE)": float(rmse_linear),
}

#calling the results to view it
linear_regression_metrics


{'Mean Absolute Error (MAE)': 124.31001764922824,
 'Mean Squared Error (MSE)': 27446.31014130236,
 'Root Mean Squared Error (RMSE)': 165.66927941324053}

In [14]:
# Training Polynomial Features (OF degree 4)

poly_transformer = PolynomialFeatures(degree=4, include_bias=False)
X_train_poly = poly_transformer.fit_transform(X_train_scaled)
X_test_poly = poly_transformer.transform(X_test_scaled)

# Training the Polynomial Regression model
poly_regression_model = LinearRegression()
poly_regression_model.fit(X_train_poly, y_train)

# Predicting the test set
y_pred_poly = poly_regression_model.predict(X_test_poly)

# Evaluating the polynomial model
mae_poly = mean_absolute_error(y_test, y_pred_poly)
mse_poly = mean_squared_error(y_test, y_pred_poly)
rmse_poly = np.sqrt(mse_poly)

# Compiling the results
polynomial_regression_metrics = {
    "Mean Absolute Error (MAE)": mae_poly,
    "Mean Squared Error (MSE)": mse_poly,
    "Root Mean Squared Error (RMSE)": float(rmse_poly),
}

print(polynomial_regression_metrics)

# File paths for saving
model_path = "poly_regression_model.pkl"
scaler_path = "scaler.pkl"
transformer_path = "poly_transformer.pkl"

# Save the fitted encoder
encoder_path = '/Users/iwuan/OneDrive - hull.ac.uk/_GDrive/Artificial intelligence coursework/AI coursework/encoder.pkl'
joblib.dump(encoder, encoder_path)
print(f"Encoder saved at: {encoder_path}")

# Save the trained model, scaler, and transformer
joblib.dump(poly_regression_model, model_path)
joblib.dump(scaler, scaler_path)
joblib.dump(poly_transformer, transformer_path)

print(f"Model saved at: {model_path}")
print(f"Scaler saved at: {scaler_path}")
print(f"Polynomial transformer saved at: {transformer_path}")


{'Mean Absolute Error (MAE)': 0.4333596694355815, 'Mean Squared Error (MSE)': 0.2980194834417398, 'Root Mean Squared Error (RMSE)': 0.5459116077184472}
Encoder saved at: /Users/iwuan/OneDrive - hull.ac.uk/_GDrive/Artificial intelligence coursework/AI coursework/encoder.pkl
Model saved at: poly_regression_model.pkl
Scaler saved at: scaler.pkl
Polynomial transformer saved at: poly_transformer.pkl


In [15]:
# Neural network model

# Definining hyperparameter distribution for RandomizedSearchCV
param_dist = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'learning_rate_init': [0.001, 0.0001],  # Try smaller learning rates
    'max_iter': [500, 1000]  # Increased iterations
}

# Custom scoring function for the RandomizedSearchCV
# The randomisedsearchCv is for hyper parameter tuning. It randomly selects combinations of hyperparameter values to evaluate.
scoring = make_scorer(mean_absolute_error, greater_is_better=False)

# Randomized search with early stopping
random_search = RandomizedSearchCV(
    estimator=MLPRegressor(random_state=42, early_stopping=True),  # Enables early stopping
    param_distributions=param_dist,
    n_iter=10,  # Number of random configurations to try
    scoring=scoring,
    cv=2,  # Reduced cross-validation folds
    verbose=2,
    n_jobs=-1  # Parallel processing
)

# Fitting the RandomizedSearchCV on the training set
random_search.fit(X_train_scaled, y_train)

# This gets the best model from the search
best_model = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Predict on the test set using the best model
y_pred_best = best_model.predict(X_test_scaled)

# Evaluates the best model
mae_best = mean_absolute_error(y_test, y_pred_best)
mse_best = mean_squared_error(y_test, y_pred_best)
rmse_best = np.sqrt(mse_best)

# Compilation of results
best_model_metrics = {
    "Mean Absolute Error (MAE)": mae_best,
    "Mean Squared Error (MSE)": mse_best,
    "Root Mean Squared Error (RMSE)": float(rmse_best),
}

print("Best Model Metrics:", best_model_metrics)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
Best Parameters: {'solver': 'adam', 'max_iter': 1000, 'learning_rate_init': 0.001, 'hidden_layer_sizes': (100,), 'activation': 'tanh'}
Best Model Metrics: {'Mean Absolute Error (MAE)': 13.849057701872722, 'Mean Squared Error (MSE)': 698.9158795669115, 'Root Mean Squared Error (RMSE)': 26.437017221443714}
